## Create Financial Valuation Using Discounted Cash Flow Analysis

## References

1. [Investopedia](http://i.investopedia.com/inv/pdf/tutorials/dcf.pdf)
2. [...](https://.com)

## Equation

$$ DCF = \frac{FCFF_1}{(1 + r) ^ 1} + \frac{FCFF_2}{(1 + r) ^ 2} + ... + \frac{FCFF_n}{(1 + r) ^ n} $$

Where FCFF stands for unlevered free cash flow for a given year ($FCFF_1$ means free cash flow in year 1, and so on); and $r$ stands for the discount rate.

$ FCFF = EBITDA - CAPEX - \Delta NWC - tax\_expenses $

Where:
1. EBITDA = earnings before interest, depreciation, and amortization, or pretax income,
2. CAPEX = capital expenditure, or fixed assets,
3. $ \Delta $ NWC = changes in net working capital, which can be derived by subtracting total current assets and total current liabilities in current and previous years,
4. tax_expenses is already self-explanatory (hopefully).

## Load Data

In [1]:
import os
import pandas
import requests
import string
from functools import partial, reduce
from bs4 import BeautifulSoup

In [2]:
PATH = os.getcwd() + '/sample-output/'

In [3]:
def get_dataframe(year):
    
    file_name = [x for x in os.listdir('sample-output/') if x.endswith('{}.csv'.format(year))]
    
    dfs = []
    for f in file_name:
        dfs.append(pandas.read_csv(PATH + f))
        
    return reduce(partial(pandas.merge, on = 'ticker_code'), dfs)

In [4]:
dfs = []
for year in (2017, 2018):
    dfs.append(get_dataframe(year))

In [5]:
df = pandas.concat(dfs, sort = False).reset_index(drop = True)

## Calculate Free Cash Flow

In [6]:
def get_free_cash_flow(year):
    
    # select necessary variables
    # sort by ticker code in ascending order
    # reset index
    df_nwc = df[['year', 'ticker_code', 'total_current_assets', 'total_current_liabilities']]\
    .sort_values(by=['ticker_code'], ascending=True)\
    .reset_index(drop=True)
    
    # add net working capital as a column
    df_nwc['net_working_capital'] = df_nwc['total_current_assets'] - df_nwc['total_current_liabilities']
    
    # changes in net working capital
    df_nwc['net_working_capital_delta'] = df_nwc.groupby('ticker_code')['net_working_capital'].diff()
    
    # filter dataframe only in year latest year
    df_nwc = df_nwc[df_nwc['year'] == year][['ticker_code', 'net_working_capital_delta']].reset_index(drop=True)
    
    # calculate free cash flow
    df_cf = df[df['year'] == 2018][['year', 'ticker_code', 'pretax_income', 'fixed_assets', 'tax_expenses']]
    df_cf = pandas.merge(df_cf, df_nwc, how = 'inner', on = 'ticker_code').reset_index(drop = True)
    df_cf['free_cash_flow'] = df_cf['pretax_income'] \
    - df_cf['fixed_assets'] \
    - df_cf['net_working_capital_delta'] \
    - df_cf['tax_expenses']
    
    return df_cf[['year', 'ticker_code', 'free_cash_flow']].dropna().reset_index(drop=True)

In [7]:
df_cf = get_free_cash_flow(2018)

## Calculate Discount Rate (r)

What we need to find out:
1. Cost of Equity 
2. Cost of Debt

### Cost of Equity

Cost of equity can obtained by calculating [Capital Asset Pricing Model](https://www.investopedia.com/terms/c/capm.asp). Following are the items:

1. Risk-free rate
2. Beta
3. Equity market risk premium (market rate subtracted by risk-free rate)

### Risk-free rate

Risk-free rate is commonly derived from T-bills rate. In Indonesia, the equivalent is [Obligasi Negara](https://www.bi.go.id/id/moneter/obligasi-negara/Default.aspx), which is traded in the secondary market. Currently there are many traded bonds with various coupon rates, so I am going to use the average rate. First, we need to scrape the table to get the coupon rate. 

In [8]:
def get_risk_free_rate():
    
    "Returns risk free rate"
    
    url = requests.get('https://www.bi.go.id/id/moneter/obligasi-negara/Default.aspx')
    soup = BeautifulSoup(url.text, 'html')
    table = soup.find('table', {'class': 'table1'})
    trs = table.find_all('tr')
    tds = []
    for t in trs:
        try:
            tds.append(t.find_all('td')[-1].text.replace(' ', ''))
        except IndexError:
            pass
    for t in range(len(tds)):
        if '\r' in tds[t]:
            tds[t] = tds[t].replace('\r', '')
        elif '\n' in tds[t]:
            tds[t] = tds[t].replace('\n', '')
        try:
            tds[t] = float(tds[t]) / 100
        except ValueError:
            pass
        
    risk_free_rate = []
    for t in tds:
        if type(t) == float:
            risk_free_rate.append(t)
    
    return sum(risk_free_rate) / len(risk_free_rate)

In [9]:
risk_free_rate = get_risk_free_rate()

### Cost of Debt

Cost of debt, or symbolized as RD, can be calculated as follows:

$$ RD = 1 - tax\_rate $$

According to [Trading Economics](https://tradingeconomics.com/indonesia/corporate-tax-rate), Indonesia's corporate tax rate is 25%. Hence, the cost of equity is 75%.

### Capital Structure (Weighted Average Cost of Capital)

## Calculate Fair Value

What we need to find out is the terminal value

$$ \frac{FPYCF * (1 + LTCFGR)}{r - LTCFGR} $$

Jeez, what's that?!?!?!

1. FPYCF = Final projected year cash flow
2. LTCGR = Long-term cash flow growth rate
3. r = discounted rate